<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/Parseo_de_Discursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs

Mounted at /content/drive
 99% 90.0M/90.8M [00:02<00:00, 57.1MB/s]
100% 90.8M/90.8M [00:02<00:00, 40.2MB/s]


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
# Cargamos la base
base = pd.read_csv('/content/Scrap PDFs.csv')

In [53]:
import pandas as pd
import re

pd.options.mode.chained_assignment = None
def generar(l):
  lineas = l[2].strip().split("\n")

  coincidencias_nombre_cargo = []
  coincidencias_discurso = []
  discurso_actual = ""

  for linea in lineas:
      if linea[0].isupper() and (":" in linea or "(" in linea):
          coincidencias_nombre_cargo.append(linea.split(":")[0].strip())
          coincidencias_discurso.append(discurso_actual)
          discurso_actual = linea.strip()
      else:
          discurso_actual += " " + linea.strip()

  # Agregar la última coincidencia de discurso
  coincidencias_discurso.append(discurso_actual)

  # Verificar si hay la misma cantidad de coincidencias para nombre/cargo y discursos
  if len(coincidencias_nombre_cargo) == len(coincidencias_discurso[1:]):
      # Crear el DataFrame de pandas
      data = {
          "Nombre/Cargo": coincidencias_nombre_cargo,
          "Discurso": coincidencias_discurso[1:]
      }
      df = pd.DataFrame(data)
      df["Discurso"] = df["Discurso"].str.strip()

  else:
      print("Error: no coincide la cantidad de coincidencias para nombre/cargo y discursos.")

  df = df[df['Nombre/Cargo'].str.contains(r'[A-Z]{2,}')]
  df['Discurso'] = df.apply(lambda row: re.sub(re.escape(row['Nombre/Cargo']) + r':\s*', '', row['Discurso']), axis=1)
  df['Cargo'] = df['Nombre/Cargo'].apply(lambda x: ' '.join(re.findall(r'\b[A-Z][A-ZÁÉÍÓÚÜÑ-]+\b', x)))
  df['Nombre'] = df['Nombre/Cargo'].apply(lambda x: re.findall(r'\((.*?)\)', x)[0] if re.findall(r'\((.*?)\)', x) else '')
  df['Link']=l[1]
  df['Legislatura']=l[1][27:29]
  df['fecha']= l[2][l[2].find('Núm.')+7:l[2].find('Pág.')].strip(' ')
  df['tipo']= l[2][:re.search('\n', l[2]).span()[0]]
  return df

In [58]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo', 'Nombre'])

In [59]:
for row in base.values:
  base2 = pd.concat([base2, generar(row)], axis=0)

In [60]:
base2=base2.reset_index(drop=True)

In [61]:
from google.colab import files
base2.to_csv('base_parseada.csv', escapechar='\\')
files.download('base_parseada.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>